# Parsbert 

### Libraries

In [1]:
!pip install transformers
! pip install nltk
import nltk
nltk.download('punkt')

     |████████████████████████████████| 3.4 MB 12.4 MB/s 
     |████████████████████████████████| 596 kB 42.4 MB/s 
     |████████████████████████████████| 3.3 MB 40.9 MB/s 
     |████████████████████████████████| 61 kB 505 kB/s 
     |████████████████████████████████| 895 kB 40.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
! wget https://raw.githubusercontent.com/language-ml/2-LM-embedding-projects/main/problem3/evaluation_IR.yml -P ./data
! wget https://github.com/language-ml/2-LM-embedding-projects/raw/main/problem3/doc_collection.zip -P ./data
! unzip ./data/doc_collection.zip -d ./data

--2021-12-27 12:25:45--  https://raw.githubusercontent.com/language-ml/2-LM-embedding-projects/main/problem3/evaluation_IR.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50854 (50K) [text/plain]
Saving to: ‘./data/evaluation_IR.yml’

evaluation_IR.yml   100%[===================>]  49.66K  --.-KB/s    in 0.004s  

2021-12-27 12:25:45 (12.2 MB/s) - ‘./data/evaluation_IR.yml’ saved [50854/50854]

--2021-12-27 12:25:45--  https://github.com/language-ml/2-LM-embedding-projects/raw/main/problem3/doc_collection.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/language-ml/2-LM-embedding-projects

### Read data

set path of data in `PATH` variable

In [3]:
PATH = './data/IR_dataset/'
YAML_PATH = './data/evaluation_IR.yml/'
YAML_PATH = YAML_PATH.rstrip('/')
PATH = PATH.rstrip('/')

store txt files into a list named `doc`

In [4]:
from nltk.tokenize import sent_tokenize

docs = []
for index in range(0, 3258):
    with open(f"{PATH}/{index}.txt", 'r', encoding='utf8') as file_reader:
      doc = file_reader.read()
      # doc = sent_tokenize(doc)
      docs.append(doc)

In [5]:
len(docs)

3258

In [6]:
import yaml

In [63]:
evaluation = {}
with open(YAML_PATH, "r") as stream:
    try:
        evaluation = dict(yaml.safe_load(stream))
    except yaml.YAMLError as exc:
        print(exc)

### Check GPU Availability

In [8]:
import torch
torch.cuda.is_available()

True

### Implementation

In [53]:
top_k = 100

In [10]:
!pip install -qU sentence-transformers

     |████████████████████████████████| 78 kB 2.0 MB/s 
     |████████████████████████████████| 1.2 MB 35.4 MB/s 


In [11]:
from sentence_transformers import models, SentenceTransformer, util

In [12]:
def load_st_model(model_name_or_path):
    word_embedding_model = models.Transformer(model_name_or_path)
    pooling_model = models.Pooling(
        word_embedding_model.get_word_embedding_dimension(),
        pooling_mode_mean_tokens=True,
        pooling_mode_cls_token=False,
        pooling_mode_max_tokens=False)
    
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    return model

In [13]:
# Load the Sentence-Transformer
model = load_st_model('m3hrdadfi/bert-zwnj-wnli-mean-tokens')

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/451M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/300 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/747k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [14]:
corpus_embeddings = model.encode(docs, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/102 [00:00<?, ?it/s]

In [17]:
queries = evaluation.keys()

In [21]:
result = []
for query in queries:
    query_embedding = model.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()
    result.append(cos_scores)

In [54]:
#We use torch.topk to find the highest 5 scores
top_results_indices = []
top_results_values = []
for cos_score in result:
  top_results_indices.append(torch.topk(cos_score, k=top_k).indices.numpy())
  top_results_values.append(torch.topk(cos_score, k=top_k).values.numpy())

In [55]:
print("Sentence embeddings:")
print(model.parameters())

Sentence embeddings:
<generator object Module.parameters at 0x7fd815194a50>


In [56]:
evaluation['آدولف هیتلر  شکست و مرگ']['relevant'][0]

341

In [65]:
fin = (top_results_indices[0] == evaluation['آدولف هیتلر  شکست و مرگ']['relevant'][0]).astype(int)

In [67]:
fin

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [58]:
!pip install import-ipynb
import import_ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=578d254d69e1f0468b3e23273e457326a79860f0670591984b34a9711f11a9fd
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb


In [64]:
import evaluation as ev

In [72]:
ev.mean_reciprocal_rank([[1, 0, 0]]) 

1.0

### Pooling Methods

In [ ]:
# Max Pooling - Take the max value over time for every dimension. 
def max_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    token_embeddings[input_mask_expanded == 0] = -1e9  # Set padding tokens to large negative value
    return torch.mean(token_embeddings, 1)[0]

In [ ]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

#### Fine Tune Model

In [ ]:
sents = []
for doc in docs:
  sents.append(nltk.sent_tokenize(doc))

In [ ]:
! pip install stanza

In [ ]:
import stanza
stanza.download('fa')
nlp = stanza.Pipeline(lang='fa', processors='tokenize')

In [ ]:
bag = list(set([item for doc in docs for item in nlp(doc).sentences]))
bag_size = len(bag)
bag_size

In [ ]:
import random

sentence_a = []
sentence_b = []
label = []

for sentence in docs:
    num_sentences = len(sentence)
    if num_sentences > 1:
        start = random.randint(0, num_sentences-2)
        # 50/50 whether is IsNextSentence or NotNextSentence
        if random.random() >= 0.5:
            # this is IsNextSentence
            sentence_a.append(sentence[start])
            sentence_b.append(sentence[start+1])
            label.append(0)
        else:
            index = random.randint(0, bag_size-1)
            # this is NotNextSentence
            sentence_a.append(sentence[start])
            sentence_b.append(bag[index])
            label.append(1)

In [ ]:
inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [ ]:
inputs.keys()

In [ ]:
inputs['labels'] = torch.LongTensor([label]).T

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

In [ ]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-6)

In [ ]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = MeditationsDataset(inputs)

In [ ]:
torch.cuda.empty_cache()
loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
from tqdm import tqdm  # for our progress bar

epochs = 10

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        print(input_ids.shape)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

In [ ]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))